In [1]:
from pyspark import SparkContext 
from pyspark import SQLContext
import sys
import os
from operator import add

## Build Spark Context

In [2]:
sc = SparkContext(appName ='Page Rank', master = 'local')
sc

<SparkContext master=local appName=Page Rank>

## Data Overview

In [3]:
path = '/Users/tjmask/Desktop/Semester 3/MapReduce/homework/hw3/'
data = sc.textFile(path+'web-BerkStan.txt')

In [4]:
data.take(10)

['1\t2',
 '1\t5',
 '1\t7',
 '1\t8',
 '1\t9',
 '1\t11',
 '1\t17',
 '1\t254913',
 '1\t438238',
 '254913\t255378']

In [5]:
def splitting(lines):
    lines = lines.split('\t')
    return (lines[0], lines[1])

In [6]:
data_splitted = data.map(splitting)
data_splitted.take(10)

[('1', '2'),
 ('1', '5'),
 ('1', '7'),
 ('1', '8'),
 ('1', '9'),
 ('1', '11'),
 ('1', '17'),
 ('1', '254913'),
 ('1', '438238'),
 ('254913', '255378')]

## Page Rank

In [15]:
def computeContribs(urls, rank):
    """Calculates URL contributions to the rank of other URLs."""
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)

In [45]:
# Loads all URLs from input file and initialize their neighbors.
links = data_splitted.groupByKey().cache()

# Loads all URLs with other URL(s) link to from input file and initialize ranks of them to one.
ranks = links.map(lambda url_neighbors: (url_neighbors[0], 1.0/685230.0))

In [46]:
from time import *
a = time()

# Calculates and updates URL ranks continuously using PageRank algorithm.
for iteration in range(10):
    # Calculates URL contributions to the rank of other URLs.
    contribs = links.join(ranks).flatMap(
        lambda url_urls_rank: computeContribs(url_urls_rank[1][0], url_urls_rank[1][1]))

    # Re-calculates URL ranks based on neighbor contributions.
    ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)


## rank them and take the top 100
ranks = ranks.top(100, key=lambda items: items[1])

print("top 100 links are as follows: ")
for (link, rank) in ranks:
    print("%s has rank: %s." % (link, rank))

b = time()
print('total time is %s second.' % (b-a))

top 100 links are as follows: 
272919 has rank: 5135.337685312974.
438238 has rank: 3491.0960312742745.
571448 has rank: 1864.6376899641693.
601656 has rank: 1602.690114457544.
319209 has rank: 1399.2280593912456.
316792 has rank: 1332.0106563322552.
184094 has rank: 1282.3453401223012.
401873 has rank: 1180.345170594501.
571447 has rank: 1148.8928422268498.
284306 has rank: 1138.1227323344524.
768 has rank: 1082.2358974713154.
927 has rank: 1010.7247942388504.
66244 has rank: 973.5471932820052.
68949 has rank: 972.971066110794.
68948 has rank: 965.9204723030336.
95552 has rank: 954.2712527121563.
77284 has rank: 954.2712527121563.
86237 has rank: 954.2712527121563.
95551 has rank: 954.2712527121563.
68947 has rank: 954.2712527121563.
96070 has rank: 954.2712527121563.
86238 has rank: 954.2712527121563.
68946 has rank: 954.2712527121563.
86239 has rank: 954.2712527121563.
66909 has rank: 954.2712527121563.
184142 has rank: 781.19488314776.
299039 has rank: 770.664978342663.
571451 has 